In [1]:
import pandas as pd
import time
from tqdm import tqdm
tqdm.pandas()


# Функция для проверки технической возможности.
# Необходимо передать очередь, was_rep, шаги.
def teh_v(o, was_rep, route):
    value = None
    # Статус "Не проверялось".
    if was_rep == 0:
        return "Didn't check"
    else:
        route = route.split(',')
        o = str(o)
        # Проверка, находится ли очередь в словаре "Есть техническая возможность".
        if o in est_tehv:
            for i in route:
                if i in est_tehv[o]:
                    value = 1
        if o in net_tehv:
            for i in route:
                if i in net_tehv[o]:
                    value = 0
        if value == 1 or value == 0:
            return value
        else:
            return "Don't know"


l_f = input('Введите название и расширение файла БД на чтение: ')
# r_f = input('Введите название и расширение файла со статусами на чтение: ')
r_f = 'Status_dict.csv'
start_time = time.time()

net_tehv = dict()
est_tehv = dict()
# my_columns = ['my_date', 'uniqueid', 'queue', 'last_step', 'route', 'billsec', 'client_status', 'otkaz', 'directory', 'server_number', 'city_c', 'ptv_c', 'marker', 'was_repeat', 'phone', 'teh_vozmozhnost', 'region', 'status', 'alive']

print(f'Создание словаря с технической возможностью начато в: {time.strftime("%X")}')
with open('Status_dict.csv') as status:
    for now in status:
        now = now.split(';')
        o, step, stat = now[0], now[1], now[2]
        if stat == 'net_teh_vozmozhnosti':
            if o in net_tehv:
                net_tehv[o].add(step)
            else:
                net_tehv[o] = set()
                net_tehv[o].add(step)
        elif stat == 'est_teh_vozmozhnost':
            if o in est_tehv:
                est_tehv[o].add(step)
            else:
                est_tehv[o] = set()
                est_tehv[o].add(step)
print(f'Словарь создан в: {time.strftime("%X")}')
print(f'Ушло времени: {round(time.time() - start_time, 3)} сек.')

# left = pd.read_csv(l_f, sep=';', nrows=100000)
left = pd.read_csv(l_f, sep=';')
city = pd.read_excel('Макрорегионы.xlsx')
right = pd.read_csv(r_f, sep=';')

left['teh_vozmozhnost'] = left.progress_apply(lambda x: teh_v(x['ochered'], x['was_repeat'], x['route']), axis=1)
left = pd.merge(left, city[['city_c', 'Область']], left_on = 'city_c', right_on = 'city_c', how='left')

result = pd.merge(left, right, left_on=['ochered', 'last_step'], right_on=['ochered', 'last_step'], how='left')
# result = pd.DataFrame(result, columns = my_columns)
result['alive'] = result['alive'].fillna(1)
result = result.astype({'alive': 'int64'})
result.rename(columns = {'Область' : 'region'}, inplace = True)

result.info()

result.to_csv('F_result.csv', index=False, sep=';', encoding='utf-8')

print(f'Создан файл: F_result.csv {time.strftime("%X")}')
print(f'Время обработки и создания файлов: {round(time.time() - start_time, 3)} сек.')


Введите название и расширение файла БД на чтение: Total_calls.csv
Создание словаря с технической возможностью начато в: 11:29:13
Словарь создан в: 11:29:13
Ушло времени: 0.024 сек.


100%|██████████| 20995352/20995352 [18:56<00:00, 18477.55it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20995352 entries, 0 to 20995351
Data columns (total 19 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   my_date          object
 1   uniqueid         object
 2   ochered          int64 
 3   last_step        int64 
 4   route            object
 5   billsec          int64 
 6   client_status    object
 7   otkaz            object
 8   directory        object
 9   server_number    int64 
 10  city_c           object
 11  ptv_c            object
 12  marker           int64 
 13  was_repeat       int64 
 14  phone            int64 
 15  teh_vozmozhnost  object
 16  region           object
 17  status           object
 18  alive            int64 
dtypes: int64(8), object(11)
memory usage: 3.1+ GB
Время обработки и создания файлов: 2140.224 сек.
Создан файл: F_result.csv 12:04:53
